<a href="https://colab.research.google.com/github/mclabs74/inference_nbs/blob/dev/yolov5/pretrained_pil_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

based on https://github.com/ultralytics/yolov5/issues/36

In [ ]:
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [1]:
import cv2
import torch
from PIL import Image
import torch
torch.hub.set_dir("/tmp/model")
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to ./model/master.zip


  0%|          | 0.00/755k [00:00<?, ?B/s]

requirements: Pillow>=8.0.0 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2

requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: torchvision>=0.8.1 not found and is required by YOLOv5, attempting auto-update...

requirements: 3 packages updated per /content/model/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2021-8-31 torch 1.9.0+cu102 CPU



  0%|          | 0.00/14.1M [00:00<?, ?B/s]

Fusing layers... 


Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
!pip install -q git+https://github.com/matthewchung74/inference_params.git

In [ ]:
import torch.nn.functional as F
import torch
import io
import base64
import json

from inference_params.inference_params import inference_test, FieldType, inference_predict

# text input will be the label for the app input
input = {"image input": FieldType.PIL}
# result will be the label for the app output
output = {"result image": FieldType.PIL, "result text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = params['image input']
    results = model(img)  # inference
    results.imgs # array of original images (as np array) passed to model for inference
    results.render()  # updates results.imgs with boxes and labels
    pil_image = Image.fromarray(results.imgs[0])
    records = results.pandas().xyxy[0].to_csv()
    return {"result image": pil_image, "result text": records}

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/zidane.jpg', 'zidane.jpg')
    img = Image.open('zidane.jpg')  # PIL image

    from inference_params.inference_params import in_colab
    from matplotlib.pyplot import imshow
    import numpy as np

    from google.colab.patches import cv2_imshow
    result, duration = predict({'image input': img})
    result_image = result['result image']
    result_text = result['result text']

    print(result_text)
    display(result_image)

In [ ]:
from inference_params.inference_params import in_colab

if in_colab():
    inference_test(predict_func=predict, params={'image input': img})